**CPLEX installation guide:** Add following line to your `.bash_profile` (for macOS) after installing IBM ILOG CPLEX Optimization Studio locally using installer from IBM. (note the path will change if you are using different python or OS)

```export PYTHONPATH="/Applications/CPLEX_Studio1210/cplex/python/3.6/x86-64_osx"```

In [1]:
import cplex
print(cplex.__path__)  # make sure the path is the one we pick above

['C:\\Program Files\\IBM\\ILOG\\CPLEX_Studio1210\\cplex\\python\\3.7\\x64_win64\\cplex']


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from util_io import (
    init, finalize, dump_conf, assigned_day_to_family_on_day, assigned_day_to_occupancy
)
from util_cost import (
    cal_total, n_people, family_id_choice_to_pref_cost, cal_total_preference, cal_total_accounting,
    nd_ndp1_to_account_penality, family_id_days_to_pref_cost
)
from util_cost import choices as family_pref
from util_check import deep_check, check_valid_all

In [3]:
from docplex.mp.model import Model

## Parameters

In [4]:
# constants #
N_families = 5000
N_days = 100
N_min_people = 125
N_max_people = 300
# constants #

# params #
path_init_conf =     '../output/m24-improved-random.csv'
path_dump_improved = '../output/m24-improved-2.csv' # output solution

num_cpu_cores = 12
#time_limit = -1 # unlimited
time_limit = 30*60  # in s

select_days = 90  # N of worst pref cost days selected
window_run = 1
new_init_update_frequency = 5
occupancy_diff = 0  # +- the occupancy of input solution for each day
occupancy_diff_low = 0  # +- the occupancy of input solution for each day
max_family_rank = 6  # maximum number of rank of the preference days for each family
use_hint = True      # use current input as hint
occupancy_count_as_variables = False  # use occupancy_counts as variable (seem faster)
redundant_occupancy_constraints = True  # use redundant constraints
min_choice_0_families = 3000   # minimum number of families that are at their choice 0
target_pref_cost = 0 # 62868
target_pref_cost_error = 0
target_pref_cost_lower = 0
target_accounting_cost = 0 # 6020.043432
target_accounting_cost_error = 0
target_accounting_cost_lower = 0
max_accounting_cost_per_day = 300

In [5]:
N_families - 62868 / 50

3742.6400000000003

In [6]:
families = range(N_families)
days = range(1, N_days + 1)
allowed_occupancy = range(N_min_people, N_max_people + 1)
possible_family_sizes = np.unique(n_people)

In [7]:
# occupancy pairs [o, o_next] limited by accounting cost
viable_nd_ndp1 = nd_ndp1_to_account_penality <= max_accounting_cost_per_day

In [8]:
# Possible choice for the family
# last choice is any day that is not on the family's preferred days
N_choices_ori = family_id_choice_to_pref_cost.shape[1]
N_choices = min(N_choices_ori, max_family_rank)
print('Limit family choice rank:', N_choices_ori, '->', N_choices)

Limit family choice rank: 11 -> 6


In [9]:
N_family_pref = min(N_choices, N_choices_ori - 1)
print(N_family_pref)

6


In [10]:
# day to dictionary of families who choose this day with value as preference rank
days_family_prefered = [{} for day in range(N_days+1)]  # day = 0 should not be used
for family, pref in enumerate(family_pref):
    for rank, day in enumerate(pref):
        if rank < N_family_pref:
            days_family_prefered[day][family] = rank

## MIP progress listener (need initial score)

In [11]:
from docplex.mp.progress import TextProgressListener
from docplex.mp.progress import ProgressClock
from docplex.mp.progress import SolutionRecorder

See http://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.progress.html#docplex.mp.progress.ProgressClock for progress clock parameters meaning

In [12]:
class MyProgressListener(SolutionRecorder):
    def __init__(self, initial_score=999999, clock=ProgressClock.Gap, absdiff=None, reldiff=None):
        super(MyProgressListener, self).__init__(clock, absdiff, reldiff)
        self.current_objective = initial_score
        
    def notify_solution(self, sol):
        if self.current_progress_data.current_objective >= self.current_objective:
            return
        print ('Improved solution')
        super(MyProgressListener, self).notify_solution(sol)
        self.current_objective = self.current_progress_data.current_objective
        assigned_day_new_raw = np.ones(N_families, dtype='int32') * -1
        for family, choice in sol.get_value_dict(assignment_matrix, keep_zeros=False):
            assigned_day_new_raw[family] = family_pref[family, choice] if choice < N_family_pref else -1
        solution = pd.DataFrame(data=families, columns = ['family_id'])
        solution['assigned_day'] = assigned_day_new_raw
        #score = cost_function(preds)
        #print('Score: ' + str(score))        
        solution.to_csv(path_dump_improved, index=False)
        
    def get_solutions(self):
        return self._solutions

## other function 

In [13]:
def distribute_unpreferred_day(assigned_day, unpreferred_day_counts_sol, n_people):
    """ Distribute unpreferred day to each family who has -1 day assigned """
    assigned_day = assigned_day.copy()
    unpreferred_days = {size: [] for size in possible_family_sizes}
    for size in possible_family_sizes:
        for day, quota in enumerate(unpreferred_day_counts_sol[size]):
            unpreferred_days[size] = unpreferred_days[size] + [day] * quota
    unpreferred_day_headers = {size: 0 for size in possible_family_sizes}
    for family, (day, size) in enumerate(zip(assigned_day, n_people)):
        if day == -1:
            assigned_day[family] = unpreferred_days[size][unpreferred_day_headers[size]]
            unpreferred_day_headers[size] += 1
    return assigned_day

## Load initial

In [14]:
assigned_day, family_on_day, occupancy = init(path_conf=path_init_conf)
print('Init config:')
try:
    is_valid = deep_check(assigned_day, family_on_day, occupancy)
except:
    is_valid = False
initial_score = cal_total(assigned_day, occupancy)
print('Valid solution: ', is_valid)
print('Total score:    ', initial_score)
print('Preference cost:', cal_total_preference(assigned_day))
print('Accounting cost:', cal_total_accounting(occupancy))

# init_occupancy_counts = {o: 0 for o in allowed_occupancy}
# for o in occupancy[1:-1]:
#     init_occupancy_counts[o] += 1
# init_occupancy_counts = pd.Series(init_occupancy_counts)

# accounting_cost_per_day = pd.Series({
#     day: nd_ndp1_to_account_penality[occupancy[day], occupancy[day+1]].astype('float32')
#     for day in days
# })

### Day Family Choice rank

# assigned_day_pref_rank = (family_pref == assigned_day.reshape(-1, 1)).argmax(axis=1)

# pd.Series(assigned_day_pref_rank).value_counts()

# df_assigned_day = pd.DataFrame({'day': assigned_day, 'rank': assigned_day_pref_rank, 'n': n_people})

# df_day_pref_rank = df_assigned_day.groupby(['day', 'rank'])[['n']].sum().reset_index().pivot(index='day', columns='rank', values='n').fillna(0).astype('int32')

Read initial configs...
Read config completed.
Init config:
deep check: everything looks fine.
Valid solution:  True
Total score:     68924.84720657553
Preference cost: 63308
Accounting cost: 5616.847206575537


In [15]:
best_score = initial_score

## Loop

In [16]:
from itertools import product

In [17]:
k = 0
for i, p in product(range(window_run), range(int(N_days / select_days)+1)):
    if k % new_init_update_frequency == 0:
        assigned_day_o, family_on_day_o, occupancy_o = init(path_conf=path_init_conf)
        print('Init config:', k)
        try:
            is_valid = deep_check(assigned_day_o, family_on_day_o, occupancy_o)
        except:
            is_valid = False
        new_initial_score = cal_total(assigned_day_o, occupancy_o)
        if new_initial_score < best_score and is_valid:
            print('Total score:    ', new_initial_score)
            print('Preference cost:', cal_total_preference(assigned_day_o))
            print('Accounting cost:', cal_total_accounting(occupancy_o))
            print('using new conf')
            best_score = new_initial_score
            assigned_day = assigned_day_o
            family_on_day = family_on_day_o
            occupancy = occupancy_o
    k += 1

    families_cost = family_id_days_to_pref_cost[np.arange(N_families), assigned_day]
    df_family = pd.DataFrame({'day': assigned_day, 'cost': families_cost})
    day_pref_cost = df_family.groupby('day')[['cost']].sum()
    iw = int(i / select_days)
    expensive_days = day_pref_cost.sort_values('cost', ascending=False).iloc[
        i%select_days + p * (select_days + iw) : i%select_days + (p+1) * (select_days + iw)].index.values
    print('[', i, ',', p,']', expensive_days)
    if len(expensive_days) == 0:
            continue
    
    # limit the occupancy choice to +- occupancy_diff of current solution
    search_occupancy = {}
    for day in days:
        if day in expensive_days:
            search_occupancy[day] = range(N_min_people, N_max_people+1)
        elif occupancy[day] == N_min_people:
            search_occupancy[day] = range(N_min_people, occupancy[day] + occupancy_diff_low + 1)
        else:
            search_occupancy[day] = range(max(occupancy[day] - occupancy_diff, N_min_people),
                                          min(occupancy[day] + occupancy_diff, N_max_people) + 1)

    # for i, x in search_occupancy.items():
    #     print(i, x)

    ## DOCplex model

    solver = Model('')

    if num_cpu_cores > 0:
        solver.context.cplex_parameters.threads = num_cpu_cores
        print('Set num threads:', num_cpu_cores)
    print('Num treads:', solver.context.cplex_parameters.threads)
    if time_limit > 0:
        print('Set time limit:', solver.set_time_limit(time_limit))

    solver.parameters.mip.tolerances.mipgap = 0  # set mip gap to 0

    ## Variables

    # Variables
    # assignment matrix[family, pref_rank]
    assignment_matrix = solver.binary_var_matrix(families, range(N_choices), 'x')

    # unpreferred_day_counts[day, size]
    if N_choices_ori <= N_choices:
        print('using unpreferred day counts')
        ub = int(N_max_people / possible_family_sizes.min())
        unpreferred_day_counts = solver.integer_var_matrix(days, possible_family_sizes, lb=0, ub=ub, name='d')
        print(len(unpreferred_day_counts))    

    # Occupancy matrix [day, N_d, N_d+1]
    occupancy_keys_list = []
    for day in days:
        if day < N_days:
            for o in search_occupancy[day]:
                for o_next in search_occupancy[day + 1]:
                    if viable_nd_ndp1[o, o_next]:
                        occupancy_keys_list.append((day, o, o_next))
        else:
            # last day
            for o in search_occupancy[day]:
                if viable_nd_ndp1[o, o]:
                    occupancy_keys_list.append((day, o))
    occupancy_matrix = solver.binary_var_dict(occupancy_keys_list, name='o')

    ## Constraints

    ### constraint 1: each family only take one day (choice)

    # Constraints
    # constraint 1: each family only take one day (choice)
    solver.add_constraints_([
        solver.sum([assignment_matrix[family, c] for c in range(N_choices)]) == 1 
        for family in families
    ])

    ### constraint: choices limit

    if min_choice_0_families > 0:
        solver.add_constraint_(
            solver.sum([assignment_matrix[family, 0] for family in families]) >= min_choice_0_families
        )

    ### occupancy counts

    # constraint 2: each day can only have 125-300 people

    # occupancy count [intermediate variables]

    if occupancy_count_as_variables:
        lbs = [min(search_occupancy[day]) for day in days]
        ubs = [max(search_occupancy[day]) for day in days]
        occupancy_counts = solver.integer_var_dict(days, lb=lbs, ub=ubs, name='oc')

        for day in days:
            # find those family who like this day
            family_prefered = days_family_prefered[day]
            solver.add_constraint_(
                occupancy_counts[day] == (
                    solver.sum(
                        [assignment_matrix[family, pref_rank] * n_people[family] 
                         for family, pref_rank in family_prefered.items()]
                    ) + (
                        solver.sum(
                            [unpreferred_day_counts[day, size] * size for size in possible_family_sizes]
                        ) if N_choices >= N_choices_ori else 0
                    )
                )
            )
    else:
        occupancy_counts = {}
        for day in days:
            # find those family who like this day
            family_prefered = days_family_prefered[day]
            occupancy_counts[day] = (
                solver.sum(
                    [assignment_matrix[family, pref_rank] * n_people[family] 
                     for family, pref_rank in family_prefered.items()]
                ) + (
                    solver.sum(
                        [unpreferred_day_counts[day, size] * size for size in possible_family_sizes]
                    ) if N_choices >= N_choices_ori else 0
                )
            )

    if not occupancy_count_as_variables:
        for day in days:
            solver.add_range(min(search_occupancy[day]), 
                             occupancy_counts[day], 
                             max(search_occupancy[day]))

    ### constraint 3: unpreferred day family count conservation for each family size

    # constraint 3: unpreferred day family count conservation for each family size

    family_size_to_family_ids = {
        size: np.where(n_people == size)[0] for size in possible_family_sizes
    }

    if N_choices >= N_choices_ori:
        solver.add_constraints_([
            solver.sum([assignment_matrix[family, N_choices - 1]
                        for family in family_size_to_family_ids[size]])
            == solver.sum([unpreferred_day_counts[day, size] for day in days])
            for size in possible_family_sizes
        ])

    ### Occupancy boolean matrix normalization

    # occupancy boolean matrix normalization
    # each day only take 1 occupancy value
    for day in days:
        if day < N_days:
            occupancy_normalization = solver.sum([
                occupancy_matrix[day, o, o_next] 
                for o in search_occupancy[day]
                for o_next in search_occupancy[day + 1]
                if viable_nd_ndp1[o, o_next]
            ])
        else:
            occupancy_normalization = solver.sum([
                occupancy_matrix[day, o] 
                for o in search_occupancy[day]
                if viable_nd_ndp1[o, o]
            ])
        solver.add_constraint_(occupancy_normalization == 1)

    ### constrain 4: link occupancy boolean matrix to occupancy count

    for day in days:
        if day < N_days:
            sum_from_occupancy_matrix = solver.sum([
                occupancy_matrix[day, o, o_next] * o 
                for o in search_occupancy[day]
                for o_next in search_occupancy[day + 1]
                if viable_nd_ndp1[o, o_next]
            ])
        else:
            sum_from_occupancy_matrix = solver.sum([
                occupancy_matrix[day, o] * o 
                for o in search_occupancy[day]
                if viable_nd_ndp1[o, o]            
            ])
        solver.add_constraint_(occupancy_counts[day] == sum_from_occupancy_matrix)

    # next day occupancy consistency
    solver.add_constraints_([
        occupancy_counts[day + 1] == solver.sum([
            occupancy_matrix[day, o, o_next] * o_next 
            for o in search_occupancy[day]
            for o_next in search_occupancy[day + 1]
            if viable_nd_ndp1[o, o_next]            
        ])
        for day in days if day < N_days
    ])

    # redudant constraints
    if redundant_occupancy_constraints:
        for day in days:
            if day + 1 < N_days:
                solver.add_constraints_([
                    solver.sum([
                        occupancy_matrix[day, o_other, o] 
                        for o_other in search_occupancy[day] if viable_nd_ndp1[o_other, o]
                    ]) == solver.sum([
                        occupancy_matrix[day + 1, o, o_other]
                        for o_other in search_occupancy[day + 2] if viable_nd_ndp1[o, o_other]
                    ])
                    for o in search_occupancy[day + 1]
                ])
        solver.add_constraints_([
            solver.sum([
                occupancy_matrix[N_days - 1, o_other, o] 
                for o_other in search_occupancy[N_days - 1] if viable_nd_ndp1[o_other, o]
            ]) == occupancy_matrix[N_days, o] if viable_nd_ndp1[o, o] else 0
            for o in search_occupancy[N_days]
        ])

    ### Preference cost

    family_pref_cost = solver.sum([
        assignment_matrix[family, c] * family_id_choice_to_pref_cost[family, c]
        for family in families for c in range(1, N_choices)
    ])

    if target_pref_cost > 0:
        if target_pref_cost_error > 0:
            print('Limit preference cost in range')
            solver.add_range(
                target_pref_cost - target_pref_cost_error,
                family_pref_cost,
                target_pref_cost + target_pref_cost_error
            )
        else:
            print('Limit preference cost exactly')
            solver.add_constraint_(family_pref_cost == target_pref_cost)
    elif target_pref_cost_lower > 0:
        print('Lower bound preference cost')
        solver.add_constraint_(family_pref_cost >= target_pref_cost_lower)

    ### Accounting cost

    accounting_cost = (
        solver.sum([
            occupancy_matrix[day, o, o_next] * nd_ndp1_to_account_penality[o, o_next]
            for day in days if day < N_days
            for o in search_occupancy[day] for o_next in search_occupancy[day + 1]
            if viable_nd_ndp1[o, o_next] and o > N_min_people
        ]) +
        solver.sum([
            occupancy_matrix[N_days, o] * nd_ndp1_to_account_penality[o, o]
            for o in search_occupancy[N_days]
            if viable_nd_ndp1[o, o] and o > N_min_people  
        ])
    )

    if target_accounting_cost > 0:
        if target_accounting_cost_error > 0:
            print('Range limit accounting cost')
            solver.add_range(
                target_accounting_cost - target_accounting_cost_error,
                accounting_cost,
                target_accounting_cost - target_accounting_cost_error
            )
    elif target_accounting_cost_lower > 0:
        print('Lower bound accounting cost')
        solver.add_constraint_(accounting_cost >= target_accounting_cost_lower)

    # ==== Objective ====
    solver.minimize(family_pref_cost + accounting_cost)  # family_pref_cost + 

    ## Hint

    if use_hint:
        print('Using hint!')

        from docplex.mp.solution import SolveSolution
        var_value_map = {}

        for family in families:
            for c in range(N_choices):
                var_value_map[assignment_matrix[family, c]] = float(
                    assigned_day[family] == family_pref[family, c]
                )
        for day in days:
            if day < N_days:
                for o in search_occupancy[day]:
                    for o_next in search_occupancy[day + 1]:
                        if viable_nd_ndp1[o, o_next]:
                            var_value_map[occupancy_matrix[day, o, o_next]] = float(
                                (occupancy[day] == o) and (occupancy[day + 1] == o_next)
                            )
                        else:
                            assert not ((occupancy[day] == o) and (occupancy[day + 1] == o_next)), \
                            'Hint not valid at (%i, %i, %i)'%(day, o, o_next)
        for o in search_occupancy[N_days]:
            if viable_nd_ndp1[o, o]:
                var_value_map[occupancy_matrix[N_days, o]] = float(occupancy[N_days] == o)
            else:
                assert not (occupancy[N_days] == o), \
                'Hint not valid at (%i, %i, %i)'%(N_days, o, o)

        if occupancy_count_as_variables:
            for day in days:
                var_value_map[occupancy_counts[day]] = float(occupancy[day])

        init_solution = SolveSolution(solver, var_value_map)
        solver.add_mip_start(init_solution)

    ## Solve

    # print progress
    my_progress_listener = MyProgressListener(initial_score=best_score, clock=ProgressClock.Objective)  
    # other choice: ProgressClock.Gap
    solver.add_progress_listener(
    #     TextProgressListener(clock=ProgressClock.Gap)
        my_progress_listener
    )  


    print('N of variables (binary, int):', solver.number_of_variables, 
          '(', solver.number_of_binary_variables, ',', solver.number_of_integer_variables, ')')
    print('N of constraints:', solver.number_of_constraints)
    print('Time limit:', solver.get_time_limit())

    # Solve
    sol = solver.solve(log_output=True)

    if sol is None:
        sol = my_progress_listener.get_solutions()[-1]

    print('Solution status:', solver.get_solve_status())
    print('Total cost:', sol.objective_value, sol.get_objective_value())
    print("Time:", '%.3f' % solver.get_solve_details().time, "s")

    ## Get Solution

    assigned_day_new_raw = np.ones(assigned_day.shape, dtype='int32') * -1
    for family, choice in sol.get_value_dict(assignment_matrix, keep_zeros=False):
        assigned_day_new_raw[family] = family_pref[family, choice] if choice < N_family_pref else -1

    if N_choices >= N_choices_ori:
        unpreferred_day_counts_sol_dict = sol.get_value_dict(unpreferred_day_counts)
        unpreferred_day_counts_sol = {
            size: [0]+[int(unpreferred_day_counts_sol_dict[day, size]) for day in days]
            for size in possible_family_sizes
        }

        print('Unpreferred families slots:')
        {size: sum(counts) for size, counts in unpreferred_day_counts_sol.items()}

    if N_choices >= N_choices_ori:
        assigned_day_new = distribute_unpreferred_day(assigned_day_new_raw, unpreferred_day_counts_sol, n_people)
    else:
        assigned_day_new = assigned_day_new_raw

    print('N family unpreferred assigned:', (~(assigned_day_new == assigned_day_new_raw)).sum())

    family_on_day_new = assigned_day_to_family_on_day(assigned_day_new)
    occupancy_new = assigned_day_to_occupancy(assigned_day_new)

    try:
        is_valid = deep_check(assigned_day_new, family_on_day_new, occupancy_new)
    except:
        is_valid = False
    new_score = cal_total(assigned_day_new, occupancy_new)
    print('Valid solution: ', is_valid)
    print('Total score:    ', new_score, '(', new_score - best_score, ')')
    print('Preference cost:', cal_total_preference(assigned_day_new))
    print('Accounting cost:', cal_total_accounting(occupancy_new))
    
    # end the model
    solver.end()
    
    # update
    if new_score < best_score:
        dump_conf(assigned_day_new, path_dump_improved)
        
        best_score = new_score
        assigned_day = assigned_day_new
        family_on_day = family_on_day_new
        occupancy = occupancy_new

Read initial configs...
Read config completed.
Init config: 0
deep check: everything looks fine.
[ 0 , 0 ] [ 70  63   6  69  90  76  62  97  13  30  72 100   9  64  55  28  83  92
  34  93  65  98  79  84  71  21   2  15  20   8   7  16  41  99  23  78
  22  35  14  86  77  29  49  85  42  50  91  56  48  27  36  38  80  53
  57  94  18  87  73  37  32  44  74  67  51  43   3  59  52  26  68   4
  47  81  31  39  66  45  25  58  46  95  10  88  24  40  61  11   5  17]
Set num threads: 12
Num treads: threads:int(12)
Set time limit: None
Using hint!
N of variables (binary, int): 1142859 ( 1142859 , 0 )
N of constraints: 21249
Time limit: 1800.0
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 12
CPXPARAM_RandomSeed                              201903125
CPXPARAM_TimeLimit                               1800
CPXPARAM_MIP_Tolerances_MIPGap                   0
Presolve time = 0.13 sec. 

     65    34    68822.3781   825    68924.8472    68759.2744    82772    0.24%
     66    50    68826.6391   801    68924.8472    68759.2744   105132    0.24%
Elapsed time = 675.84 sec. (1058334.91 ticks, tree = 5.85 MB, solutions = 1)
     70    42    68778.7698   955    68924.8472    68759.2744    95660    0.24%
     74    52    68826.7928   795    68924.8472    68759.2744   105497    0.24%
     78    61    68898.3734   530    68924.8472    68759.2744   113006    0.24%
     81    64    68898.6851   509    68924.8472    68759.2744   113427    0.24%
     86    53    68827.3537   778    68924.8472    68759.2744   106084    0.24%
     90    54    68827.5012   793    68924.8472    68759.2744   106256    0.24%
     96    49    68898.5315   467    68924.8472    68759.2744   103625    0.24%
    101    50    68899.3983   490    68924.8472    68759.2744   103992    0.24%
    104    51    68899.2823   475    68924.8472    68759.2744   104240    0.24%
    106    53    68899.4192   476    68924.

    938   744    68795.7173  1007    68924.8472    68764.8125   370807    0.23%
    940   774    68772.2945  1158    68924.8472    68764.8125   390782    0.23%
    943   828    68819.2539   803    68924.8472    68764.8125   405604    0.23%
    945   830    68819.3603   798    68924.8472    68764.8125   406077    0.23%
    951   827    68795.9655  1019    68924.8472    68764.8125   405349    0.23%
    956   834    68809.3442   889    68924.8472    68764.8125   419806    0.23%
    959   835    68809.8606   836    68924.8472    68764.8125   420438    0.23%
    964   836    68809.9723   871    68924.8472    68764.8125   420935    0.23%
    966   846    68820.4114   790    68924.8472    68764.8125   432929    0.23%
Elapsed time = 1050.48 sec. (1316406.39 ticks, tree = 215.65 MB, solutions = 1)
    970   838    68811.2992   833    68924.8472    68764.8125   421923    0.23%
    974   846    68820.3559   817    68924.8472    68764.8125   433375    0.23%
    981   849    68823.1399   830    689

   1716  1460    68833.1360   834    68924.8472    68764.8125   849648    0.23%
   1721  1464    68833.4478   845    68924.8472    68764.8125   850186    0.23%
   1727  1491    68833.7916   822    68924.8472    68764.8125   906238    0.23%
   1734  1496    68834.2724   831    68924.8472    68764.8125   906828    0.23%
   1739  1499    68834.7020   848    68924.8472    68764.8125   907486    0.23%
Elapsed time = 1772.78 sec. (1759948.64 ticks, tree = 435.71 MB, solutions = 1)
   1743  1502    68834.9572   842    68924.8472    68764.8125   907971    0.23%
   1750  1505    68835.4096   836    68924.8472    68764.8125   908466    0.23%
   1752  1446    68781.0898  1134    68924.8472    68764.8125   870584    0.23%
   1755  1454    68800.3794  1036    68924.8472    68764.8125   891464    0.23%
   1756  1455    68784.9915  1160    68924.8472    68764.8125   892012    0.23%

GUB cover cuts applied:  1
Cover cuts applied:  816
Flow cuts applied:  37
Mixed integer rounding cuts applied:  272
Ze

In [19]:
best_score - initial_score

0.0

## Final output

In [20]:
## Output

is_improved = new_score < initial_score
if is_valid and (is_improved or (path_dump_improved != path_init_conf)):
    print('output to', path_dump_improved)
    dump_conf(assigned_day_new, path_dump_improved)

output to ../output/m24-improved-2.csv


## Debug

In [21]:
# [
#     [assignment_matrix[family, c].solution_value() for c in range(N_choices)]
#     for family in range(10)
# ]        

In [22]:
# if N_choices >= N_choices_ori:
#     print([
#         [unpreferred_day_counts[day, size].solution_value() for size in possible_family_sizes]
#         for day in range(1, 10)
#     ])